1. QA
- Dữ liệu có chính xác + Stable không?
- Phép tính cungxc cần QA -> móc từ một nguồn third-party 
2. Show account cụ thể


To-do list
- Health Evaluation (Done logic)
    - Components
    - Total Score (mỗi group of indicator là 1 điểm)
- CAPM ()
    company_finance_df['SaleGrowth (TTM)']=5/100
    company_finance_df['SalesGrowth_03Yr']=5/100
    company_finance_df['Projected SalesGrowth']=5/100
    company_finance_df['PreTaxMargin (TTM)']=10/100
    company_finance_df['Asset Quality Discount']=0 #cần tình
- Market-based pricing (done Logic)
- Final Pricing (xong CAPM và Market-based pricing là xong)

# A. Preparation

## 1. Import Library

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt


import streamlit as st
import json

import requests
from io import StringIO

import ast
from functions import functions

ModuleNotFoundError: No module named 'functions'

## 2. Set up Parameter

In [ ]:
risk_free_rate=7/100
income_tax=20/100

indicator_range_max=1.1
indicator_range_min=0.5
score_modifier_max=0.8
score_modifier_min=0.5

## 3. Functions

In [ ]:
def get_data_csv(url):
    response=requests.get(url)
    response.raise_for_status()
    data_raw = StringIO(response.text)
    data = pd.read_csv(data_raw)
    return data

def expand_financial_response_column(row):
    # Parse the JSON data from the 'response' column
    response_data = json.loads(row['response'])

    # Extract the 'Appendix' data, which we expect to be a dictionary
    appendix_data = response_data.get('Appendix', {})

    # Now we will flatten the appendix_data if it is not already a string
    if isinstance(appendix_data, str):
        # If it's a string, it might be JSON-encoded, so we attempt to parse it
        appendix_data = json.loads(appendix_data)

    # Then we extract items from this dictionary, assuming each value is just a string
    flattened_data = {k: v for k, v in appendix_data.items()}

    return pd.Series(flattened_data)

def organize_income_data(income_data):
    income_data_dict = {}
    for index, row in income_data.iterrows():
        ticker = row['ticker']
        try:
            response = json.loads(row['response'])  # Convert JSON string to dictionary
        except json.JSONDecodeError:
            print(f"Error decoding JSON for ticker {ticker} at row {index}")
            continue

        # Initialize the structure for the current ticker
        if ticker not in income_data_dict:
            income_data_dict[ticker] = {}

        # Check if 'Appendix' exists
        if 'Appendix' in response:
            # Append the appendix to the ticker's data
            income_data_dict[ticker]['Appendix'] = response['Appendix']

            # Iterate over the years and metrics in the response, excluding the Appendix
            for year, metrics in response.items():
                if year != "Appendix":
                    if year not in income_data_dict[ticker]:
                        income_data_dict[ticker][year] = {}
                    for metric_index, value in metrics.items():
                        # Use the appendix to get the metric name, if possible
                        metric_name = response['Appendix'].get(metric_index, f"Unknown Metric {metric_index}")
                        income_data_dict[ticker][year][metric_name] = value
        else:
            print(f"No 'Appendix' found for ticker {ticker} at row {index}")
            # Handle the case where there's no Appendix separately if needed
            # For example, you might want to simply copy the data as-is
            for year, metrics in response.items():
                income_data_dict[ticker][year] = metrics

    return income_data_dict

def organize_bs_data(bs_data):
    bs_data_dict = {}

    for index, row in bs_data.iterrows():
        ticker = row['ticker']
        try:
            response = json.loads(row['response'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for ticker {ticker} at row {index}: {e}")
            continue

        # Initialize the structure for the current ticker
        if ticker not in bs_data_dict:
            bs_data_dict[ticker] = {}
        # Check if 'Appendix' exists
        if 'Appendix' in response:
            # Append the appendix to the ticker's data
            bs_data_dict[ticker]['Appendix'] = response['Appendix']

            # Iterate over the years and metrics in the response, excluding the Appendix
            for key, metrics in response.items():
                if key != "Appendix":
                    if key not in bs_data_dict[ticker]:
                        bs_data_dict[ticker][key] = {}
                    for metric_index, value in metrics.items():
                        # Use the appendix to get the metric name, if possible
                        metric_name = response['Appendix'].get(metric_index, f"Unknown Metric {metric_index}")
                        bs_data_dict[ticker][key][metric_name] = value
        else:
            print(f"No 'Appendix' found for ticker {ticker} at row {index}")
            # Handle the case where there's no Appendix separately if needed
            # For example, you might want to simply copy the data as-is
            for key, metrics in response.items():
                bs_data_dict[ticker][key] = metrics

    return bs_data_dict

def organize_bs_data(bs_data):
    bs_data_dict = {}

    for index, row in bs_data.iterrows():
        ticker = row['ticker']
        try:
            response = json.loads(row['response'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for ticker {ticker} at row {index}: {e}")
            continue

        # Initialize the structure for the current ticker
        if ticker not in bs_data_dict:
            bs_data_dict[ticker] = {}
        # Check if 'Appendix' exists
        if 'Appendix' in response:
            # Append the appendix to the ticker's data
            bs_data_dict[ticker]['Appendix'] = response['Appendix']

            # Iterate over the years and metrics in the response, excluding the Appendix
            for key, metrics in response.items():
                if key != "Appendix":
                    if key not in bs_data_dict[ticker]:
                        bs_data_dict[ticker][key] = {}
                    for metric_index, value in metrics.items():
                        # Use the appendix to get the metric name, if possible
                        metric_name = response['Appendix'].get(metric_index, f"Unknown Metric {metric_index}")
                        bs_data_dict[ticker][key][metric_name] = value
        else:
            print(f"No 'Appendix' found for ticker {ticker} at row {index}")
            # Handle the case where there's no Appendix separately if needed
            # For example, you might want to simply copy the data as-is
            for key, metrics in response.items():
                bs_data_dict[ticker][key] = metrics

    return bs_data_dict

def weighted_average_roe(df, group_col, weight_col, value_col, top_n=5):
  """
  Calculates weighted average ROE within groups, considering null values.

  Args:
      df (pandas.DataFrame): The dataframe containing the data.
      group_col (str): The column name to group by (e.g., 'L1N').
      weight_col (str): The column name for weights (e.g., 'Gross Revenue').
      value_col (str): The column name for values (e.g., 'ROE').
      top_n (int, optional): The number of top companies to consider. Defaults to 5.

  Returns:
      pandas.Series: A Series containing the weighted average for each group.
  """
  def g(df):
    # Sort by group and weight column (descending) in case of nulls
    df=pd.concat([df[group_col],df[weight_col],df[value_col]],axis=1)
    df = df.sort_values(by=[group_col, weight_col], ascending=False)
    # Select top_n rows (considering nulls in weight_col)
    df = df.dropna(subset=[weight_col]).head(top_n)
    # Weighted average using Series methods
    weights = df[weight_col]
    values = df[value_col]
    return (weights * values).sum() / weights.sum()  # Avoid Series.weighted
  return df.groupby(group_col).apply(g)

def map_multiple_financial_metrics_to_df(data, mapping_dict, metric_names, df=None):
    """
    Map multiple financial metric values to a DataFrame based on given metric names.
    
    Parameters:
    - data: dict, latest financial data for each ticker.
    - mapping_dict: dict, maps variant metric names to standard metric names.
    - metric_names: list of str, the standard names of the metrics to map.
    - df: pd.DataFrame, the DataFrame to update. If None, a new DataFrame is created.
    
    Returns:
    - pd.DataFrame, updated with the new financial metric columns.
    """
    if df is None:
        df = pd.DataFrame()
    
    for metric_name in metric_names:
        for ticker, metrics in data.items():
            # Initialize a placeholder for the metric value for this ticker
            metric_value = None
            
            # Check each metric to see if it's a variant of the specified metric name
            for metric, value in metrics.items():
                if metric in mapping_dict and mapping_dict[metric] == metric_name:
                    metric_value = value  # Assuming only one relevant metric per ticker
            # Add the found value to the DataFrame
            df.loc[ticker, metric_name] = metric_value
    
    return df

def calculate_weighted_averages(df, level):
    """
    Calculates weighted averages of specified metrics by Gross Revenue for top 5 companies in each industry level.

    Args:
    - df: DataFrame containing company financials and rankings.
    - level: Industry level ('L1N', 'L2N', 'L3N', 'L4N').

    Returns:
    - DataFrame with industry name, level, and calculated weighted averages of metrics.
    """
    # Define the metrics for which to calculate weighted averages
    metrics = ['ROE', 'ROA', 'Receivable Turnover', 'Inventory Turnover', 'CA Turnover',
               'TA Turnover', 'Current Ratio', 'Interest Coverage Ratio', 'Leverage',
               'Short-term/Total Liabilities', 'P/S', 'P/B']
    
    # Filter top 5 companies based on rank within each industry level
    top_5_df = df[df[f'Rank {level}'] <= 5]
    
    # Calculate weighted averages
    weighted_avgs = top_5_df.groupby(level).apply(
        lambda x: pd.Series(
            {metric: np.average(x[metric], weights=x['Gross Revenue']) for metric in metrics}
        )
    ).reset_index()
    
    # Add industry level to the DataFrame
    weighted_avgs['Industry Level'] = level
    
    # Rename columns if necessary and reorder
    weighted_avgs = weighted_avgs.rename(columns={level: 'Industry Name'})
    weighted_avgs = weighted_avgs[['Industry Name', 'Industry Level'] + metrics]
    
    return weighted_avgs

def period_to_datetime(period):
    try:
        # For "YYYY" format
        return datetime(year=int(period.strip()), month=1, day=1)
    except ValueError:
        try:
            # For "Quý X- YYYY" format
            quarter, year = period.split('-')
            quarter_number = int(quarter.split(' ')[1])
            start_month = (quarter_number - 1) * 3 + 1
            return datetime(year=int(year.strip()), month=start_month, day=1)
        except ValueError:
            # If the period doesn't conform to expected formats, return None
            return None
        
def extract_latest_data(data_dict):
    latest_data = {}

    for ticker, periods in data_dict.items():
        valid_periods = {period: datetime for period, datetime in
                         ((period, period_to_datetime(period)) for period in periods.keys())
                         if datetime is not None}
        if valid_periods:  # Check if there are any valid periods
            latest_period = max(valid_periods, key=valid_periods.get)
            latest_data[ticker] = periods[latest_period]

    return latest_data

def build_company_metrics_dataframe(ticker, company_finance_df, final_summary_df):
    # Group the metrics columns
    global groups

    # Flatten the group structure for easier data retrieval
    metrics_columns = [metric for group in groups.values() for metric in group]

    final_dataframe_list = []

    # Retrieve the row for the input ticker to get industry levels and names
    company_row = company_finance_df.loc[company_finance_df['ticker'] == ticker]

    if company_row.empty:
        print(f"No data found for ticker {ticker}.")
        return None

    industry_info = company_row[['L1N', 'L2N', 'L3N', 'L4N']].iloc[0]

    for level, industry_name in industry_info.dropna().items():
        industry_row = final_summary_df.loc[final_summary_df['Industry Name'] == industry_name].iloc[0]
        top_companies = industry_row['Top 5 Companies'].split(", ")

        multi_index = pd.MultiIndex.from_product(
            [[level[-2]], [industry_name], ['Industry Average'] + top_companies],
            names=['Level', 'Industry', 'Company']
        )
        
        # Fetch metrics for the top companies and industry average
        metrics_data = company_finance_df.loc[company_finance_df['ticker'].isin(top_companies), metrics_columns]
        industry_average_metrics = final_summary_df.loc[final_summary_df['Industry Name'] == industry_name, metrics_columns].mean()

        metrics_data = metrics_data.round(2)
        industry_average_metrics = industry_average_metrics.round(2)

        # Create a DataFrame with MultiIndex columns for the groups and metrics
        tuples = [(group, metric) for group in groups for metric in groups[group]]
        multi_column = pd.MultiIndex.from_tuples(tuples, names=['Group', 'Metric'])

        # Initialize an empty DataFrame to hold reordered metrics with MultiIndex
        metrics_data_reordered = pd.DataFrame(columns=multi_column, index=metrics_data.index)

        # Populate the DataFrame
        for group, metrics in groups.items():
            for metric in metrics:
                metrics_data_reordered[(group, metric)] = metrics_data[metric]
        
        # Add the industry average metrics to the DataFrame
        industry_average_df = pd.DataFrame([industry_average_metrics.values], columns=pd.MultiIndex.from_tuples([(group, metric) for group in groups for metric in groups[group]]))
        combined_metrics_df = pd.concat([industry_average_df, metrics_data_reordered])

        combined_metrics_df.index = multi_index
        
        final_dataframe_list.append(combined_metrics_df)

    final_df = pd.concat(final_dataframe_list)
    return final_df

In [7]:
company_industry_df=pd.read_excel('E:\Downloads\master-main\master-main\company_industry.xlsx')
company_industry_df

,Stock,L4C,L4N,L3C,L3N,L2C,L2N,L1C,L1N
0,BSR,533,Thăm dò và sản xuất dầu khí,530,Sản xuất dầu khí,500,Dầu khí,1,Dầu khí
1,DMS,587,Nhiên liệu thay thế,580,Năng lượng thay thế,500,Dầu khí,1,Dầu khí
2,MTS,573,"Thiết bị, dịch vụ khai thác dầu mỏ",570,"Thiết bị, dịch vụ và phân phối dầu khí",500,Dầu khí,1,Dầu khí
3,OIL,533,Thăm dò và sản xuất dầu khí,530,Sản xuất dầu khí,500,Dầu khí,1,Dầu khí
4,PCN,533,Thăm dò và sản xuất dầu khí,530,Sản xuất dầu khí,500,Dầu khí,1,Dầu khí
...,...,...,...,...,...,...,...,...,...
1601,VIE,9578,Thiết bị viễn thông,9570,Công nghệ phần cứng và thiết bị,9500,Công nghệ,9000,Công nghệ
1602,VLA,9537,Công nghệ phần mềm,9530,Phần mềm và dịch vụ điện toán,9500,Công nghệ,9000,Công nghệ
1603,VTC,9578,Thiết bị viễn thông,9570,Công nghệ phần cứng và thiết bị,9500,Công nghệ,9000,Công nghệ
1604,VTE,9578,Thiết bị viễn thông,9570,Công nghệ phần cứng và thiết bị,9500,Công nghệ,9000,Công nghệ


In [8]:
income_url = 'https://raw.githubusercontent.com/penthousecompany/master/main/raw/financial_report_incomestatement_yearly_final/financial_report_incomestatement_yearly_final.csv'
income_data = get_data_csv(income_url)
income_data_dict = organize_income_data(income_data)
ticker = 'VNM'
year = '2019'

No 'Appendix' found for ticker ABW at row 0
No 'Appendix' found for ticker BCO at row 50
No 'Appendix' found for ticker ART at row 54
No 'Appendix' found for ticker BCR at row 55
No 'Appendix' found for ticker DLM at row 348
No 'Appendix' found for ticker DMS at row 349
No 'Appendix' found for ticker PRE at row 986
No 'Appendix' found for ticker THM at row 1240
No 'Appendix' found for ticker VCI at row 1430


## B. Data Transformation

## Income Statement Structure

In [9]:
income_data_dict['VNM']['Appendix']

{'0': '1. Doanh thu bán hàng và cung cấp dịch vụ',
 '1': '2. Các khoản giảm trừ doanh thu',
 '2': '3. Doanh thu thuần về bán hàng và cung cấp dịch vụ (10 = 01 - 02)',
 '3': '4. Giá vốn hàng bán',
 '4': '5. Lợi nhuận gộp về bán hàng và cung cấp dịch vụ(20=10-11)',
 '5': '6. Doanh thu hoạt động tài chính',
 '6': '7. Chi phí tài chính',
 '7': '- Trong đó: Chi phí lãi vay',
 '8': '8. Phần lãi lỗ trong công ty liên doanh, liên kết',
 '9': '9. Chi phí bán hàng',
 '10': '10. Chi phí quản lý doanh nghiệp',
 '11': '11. Lợi nhuận thuần từ hoạt động kinh doanh{30=20+(21-22) + 24 - (25+26)}',
 '12': '12. Thu nhập khác',
 '13': '13. Chi phí khác',
 '14': '14. Lợi nhuận khác(40=31-32)',
 '15': '15. Tổng lợi nhuận kế toán trước thuế(50=30+40)',
 '16': '16. Chi phí thuế TNDN hiện hành',
 '17': '17. Chi phí thuế TNDN hoãn lại',
 '18': '18. Lợi nhuận sau thuế thu nhập doanh nghiệp(60=50-51-52)',
 '19': '19. Lợi nhuận sau thuế công ty mẹ',
 '20': '20. Lợi nhuận sau thuế công ty mẹ không kiểm soát',
 '21'

In [10]:
# Use a dictionary to map the stringified appendix to a list of tickers
appendix_to_tickers = {}
appendix_to_tickers[""] = []

for ticker, data in income_data_dict.items():
    # Adjusted to check for 'appendix' key in a case-insensitive manner
    appendix_data = data.get('Appendix') or data.get('appendix')
    if appendix_data:
        appendix_key = str(appendix_data)
    else:
        appendix_key = ""
    if appendix_key not in appendix_to_tickers:
        appendix_to_tickers[appendix_key] = [ticker]
    else:
        appendix_to_tickers[appendix_key].append(ticker)

# Convert the dictionary to a list of tuples for easy viewing
appendix_list = []
for appendix, tickers in appendix_to_tickers.items():
    try:
        # Safely evaluate the appendix string to a dictionary
        appendix_dict = ast.literal_eval(appendix) if appendix else {}
        appendix_list.append((appendix_dict, tickers))
    except (ValueError, SyntaxError):
        # Handle the case where the string cannot be evaluated to a dictionary
        print(f"Could not evaluate appendix: {appendix}")
        appendix_list.append(({}, tickers))

# For demonstration, let's print the appendix_list
i = 0
for appendix, tickers in appendix_list:
    i += 1
    print(f"{i}, Appendix: {appendix}, Tickers: {tickers}")


1, Appendix: {}, Tickers: ['ABW', 'BCO', 'ART', 'BCR', 'DLM', 'DMS', 'PRE', 'THM', 'VCI']
2, Appendix: {'0': 'Thu nhập lãi ròng', '1': 'Thu nhập lãi và các khoản thu nhập tương tự', '2': 'Chi phí lãi và các khoản chi phí tương tự', '3': 'Lãi/lỗ ròng từ hoạt động dịch vụ', '4': 'Thu nhập từ hoạt động dịch vụ', '5': 'Chi phí hoạt động dịch vụ', '6': 'Lãi/lỗ ròng từ hoạt động kinh doanh ngoại hối và vàng', '7': 'Lãi/lỗ thuần từ mua bán chứng khoán kinh doanh', '8': 'Lãi/ lỗ thuần từ mua bán chứng khoán đầu tư', '9': 'Lãi thuần từ hoạt động khác', '10': 'Thu nhập hoạt động khác', '11': 'Chi phí hoạt động khác', '12': 'Thu nhập từ góp vốn, mua cổ phần', '13': 'Tổng thu nhập kinh doanh', '14': 'Lãi/ lỗ trong công ty liên kết, liên doanh', '15': 'Chi phí hoạt động', '16': 'Chi phí nhân viên', '17': 'Chi phí khấu hao', '18': 'Chi phí hoạt động khác', '19': 'Lợi nhuận ròng từ hoạt động kinh doanh trước chi phí dự phòng', '20': 'Chi phí dự phòng', '21': 'Chi phí dự phòng rủi ro tín dụng', '22': 

In [11]:
# Initialize an empty list to hold the data
data_list = []

for appendix, tickers in appendix_list:
    # Convert appendix dictionary to a string as a unique identifier
    appendix_str = str(appendix) if appendix else "No Appendix"
    # Add a dictionary for each row of data
    data_list.append({"Appendix": appendix_str, "Tickers": tickers})

# Create the DataFrame directly from the list of dictionaries
df_appendix = pd.DataFrame(data_list)

# Show the DataFrame
df_appendix

,Appendix,Tickers
0,No Appendix,"[ABW, BCO, ART, BCR, DLM, DMS, PRE, THM, VCI]"
1,"{'0': 'Thu nhập lãi ròng', '1': 'Thu nhập lãi ...","[AAS, ABB, ACB, BAB, BID, BVB, CTG, EIB, EVF, ..."
2,"{'0': '1- Thu phí bảo hiểm gốc', '1': '2- Thu ...","[ABI, BMI, BLI, BIC, BVH, MIG, PGI, PTI, VNR]"
3,{'0': '1. Doanh thu bán hàng và cung cấp dịch ...,"[AMV, ALV, APL, AAT, AAM, ACL, ABT, AGF, API, ..."
4,"{'0': 'I. DOANH THU HOẠT ĐỘNG', '1': '1.1. Lãi...","[APG, AGR, APS, BHI, BSI, BVS, CTS, CSI, DSC, ..."


In [12]:
# Step 1: Collect all unique financial metrics
financial_metrics = set()
for appendix, _ in appendix_list:
    financial_metrics.update(appendix.keys())

# Ensure the financial metrics are sorted by their key to maintain order
financial_metrics = sorted(financial_metrics, key=int)

# Step 2: Create the DataFrame with financial metrics as the row index
# The financial metrics descriptions will be filled later
df = pd.DataFrame(index=financial_metrics)

# Step 3: Populate the DataFrame with financial metric descriptions for each appendix type
for i, (appendix, tickers) in enumerate(appendix_list, start=1):
    # Column name based on the appendix number (i.e., Appendix 1, Appendix 2, ...)
    column_name = f"Appendix {i}"
    # Prepare the data for this column
    column_data = {metric: appendix.get(metric, '') for metric in financial_metrics}
    # Add the column to the DataFrame
    df[column_name] = pd.Series(column_data)

# Optionally, rename the index for clarity
df.index = [f"Metric {idx}" for idx in df.index]

df.to_excel('Income Statement Structure.xlsx')

# Balance Sheet

In [13]:
bs_url = r'E:\OneDrive\6. Business Ideas\Stock Recommendation Project (SRP)\2. Database\data\raw\financial_report_balancesheet_quarterly_final\financial_report_balancesheet_quarterly_final.csv'
bs_data=pd.read_csv(bs_url)
bs_data_dict = organize_bs_data(bs_data)

No 'Appendix' found for ticker ABW at row 12
No 'Appendix' found for ticker ART at row 54
No 'Appendix' found for ticker BCO at row 81
No 'Appendix' found for ticker BCR at row 84
No 'Appendix' found for ticker DLM at row 338
No 'Appendix' found for ticker DMS at row 344
No 'Appendix' found for ticker PRE at row 989
No 'Appendix' found for ticker THM at row 1278
No 'Appendix' found for ticker VCI at row 1418


In [14]:
import ast

# Use a dictionary to map the stringified appendix to a list of tickers
appendix_to_tickers = {}
appendix_to_tickers[""] = []

for ticker, data in bs_data_dict.items():
    # Adjusted to check for 'appendix' key in a case-insensitive manner
    appendix_data = data.get('Appendix') or data.get('appendix')
    if appendix_data:
        appendix_key = str(appendix_data)
    else:
        appendix_key = ""
    if appendix_key not in appendix_to_tickers:
        appendix_to_tickers[appendix_key] = [ticker]
    else:
        appendix_to_tickers[appendix_key].append(ticker)

# Convert the dictionary to a list of tuples for easy viewing
appendix_list = []
for appendix, tickers in appendix_to_tickers.items():
    try:
        # Safely evaluate the appendix string to a dictionary
        appendix_dict = ast.literal_eval(appendix) if appendix else {}
        appendix_list.append((appendix_dict, tickers))
    except (ValueError, SyntaxError):
        # Handle the case where the string cannot be evaluated to a dictionary
        print(f"Could not evaluate appendix: {appendix}")
        appendix_list.append(({}, tickers))

# For demonstration, let's print the appendix_list
i = 0
for appendix, tickers in appendix_list:
    i += 1
    print(f"{i}, Appendix: {appendix}, Tickers: {tickers}")

1, Appendix: {}, Tickers: ['ABW', 'ART', 'BCO', 'BCR', 'DLM', 'DMS', 'PRE', 'THM', 'VCI']
2, Appendix: {'0': 'TÀI SẢN', '1': 'A- TÀI SẢN NGẮN HẠN', '2': 'I. Tiền và các khoản tương đương tiền', '3': '1. Tiền', '4': '2. Các khoản tương đương tiền', '5': 'II. Các khoản đầu tư tài chính ngắn hạn', '6': '1. Chứng khoán kinh doanh', '7': '2. Dự phòng giảm giá chứng khoán kinh doanh', '8': '3. Đầu tư nắm giữ đến ngày đáo hạn', '9': 'III. Các khoản phải thu ngắn hạn', '10': '1. Phải thu ngắn hạn của khách hàng', '11': '2. Trả trước cho người bán ngắn hạn', '12': '3. Phải thu nội bộ ngắn hạn', '13': '4. Phải thu theo tiến độ kế hoạch hợp đồng xây dựng', '14': '5. Phải thu về cho vay ngắn hạn', '15': '6. Phải thu ngắn hạn khác', '16': '7. Dự phòng phải thu ngắn hạn khó đòi', '17': '8. Tài sản Thiếu chờ xử lý', '18': 'IV. Hàng tồn kho', '19': '1. Hàng tồn kho', '20': '2. Dự phòng giảm giá hàng tồn kho', '21': 'V.Tài sản ngắn hạn khác', '22': '1. Chi phí trả trước ngắn hạn', '23': '2. Thuế GTGT đ

In [15]:
bs_data_dict['VNM']['Quý 1- 2005']

{'TÀI SẢN': '',
 'A- TÀI SẢN NGẮN HẠN': '2,406,477,000,000',
 'I. Tiền và các khoản tương đương tiền': '500,312,000,000',
 '1. Tiền': '130,312,000,000',
 '2. Các khoản tương đương tiền': '370,000,000,000',
 'II. Các khoản đầu tư tài chính ngắn hạn': '22,800,000,000',
 '1. Chứng khoán kinh doanh': '',
 '2. Dự phòng giảm giá chứng khoán kinh doanh': '',
 '3. Đầu tư nắm giữ đến ngày đáo hạn': '',
 'III. Các khoản phải thu ngắn hạn': '748,600,000,000',
 '1. Phải thu ngắn hạn của khách hàng': '382,596,000,000',
 '2. Trả trước cho người bán ngắn hạn': '76,804,000,000',
 '3. Phải thu nội bộ ngắn hạn': '',
 '4. Phải thu theo tiến độ kế hoạch hợp đồng xây dựng': '',
 '5. Phải thu về cho vay ngắn hạn': '',
 '6. Phải thu ngắn hạn khác': '291,564,000,000',
 '7. Dự phòng phải thu ngắn hạn khó đòi': '-2,364,000,000',
 '8. Tài sản Thiếu chờ xử lý': '',
 'IV. Hàng tồn kho': '1,081,501,000,000',
 '1. Hàng tồn kho': '1,081,501,000,000',
 '2. Dự phòng giảm giá hàng tồn kho': '',
 'V.Tài sản ngắn hạn khác

In [16]:
# Initialize an empty list to hold the data
data_list = []

for appendix, tickers in appendix_list:
    # Convert appendix dictionary to a string as a unique identifier
    appendix_str = str(appendix) if appendix else "No Appendix"
    # Add a dictionary for each row of data
    data_list.append({"Appendix": appendix_str, "Tickers": tickers})

# Create the DataFrame directly from the list of dictionaries
df_appendix = pd.DataFrame(data_list)

# Step 1: Collect all unique financial metrics
financial_metrics = set()
for appendix, _ in appendix_list:
    financial_metrics.update(appendix.keys())

# Ensure the financial metrics are sorted by their key to maintain order
financial_metrics = sorted(financial_metrics, key=int)

# Step 2: Create the DataFrame with financial metrics as the row index
# The financial metrics descriptions will be filled later
df = pd.DataFrame(index=financial_metrics)

# Step 3: Populate the DataFrame with financial metric descriptions for each appendix type
for i, (appendix, tickers) in enumerate(appendix_list, start=1):
    # Column name based on the appendix number (i.e., Appendix 1, Appendix 2, ...)
    column_name = f"Appendix {i}"
    # Prepare the data for this column
    column_data = {metric: appendix.get(metric, '') for metric in financial_metrics}
    # Add the column to the DataFrame
    df[column_name] = pd.Series(column_data)

# Optionally, rename the index for clarity
df.index = [f"Metric {idx}" for idx in df.index]

df.to_excel('Balance Sheet Structure.xlsx')

In [17]:
structure_list={}
for ticker in company_industry_df['Stock']:
    if ticker in bs_data_dict.keys():
        if 'Appendix' in bs_data_dict[ticker]:
            balance_sheet_structure = bs_data_dict[ticker]['Appendix']
        else:
            balance_sheet_structure = ""
    else:
        balance_sheet_structure = ""
    if ticker in income_data_dict.keys():
        if  'Appendix' in income_data_dict[ticker]:
            income_structure = income_data_dict[ticker]['Appendix']
        else:
            income_structure = ""
    else:
        income_structure = "" 
    structure = str(balance_sheet_structure) + str(income_structure)
    if structure not in structure_list:
        structure_list[structure]=[ticker]
    else:
        structure_list[structure].append(ticker)

i=1
for structures,tickers in structure_list.items():
    Blank = 0
    if balance_sheet_structure == "":
        Blank +=1
    
    if income_structure == "":
        Blank +=1
    print(f"{i}, Blank {Blank}",tickers,structures)
    i+=1

1, Blank 0 ['BSR', 'MTS', 'OIL', 'PCN', 'PEQ', 'PLX', 'POB', 'POS', 'PPT', 'PSH', 'PTV', 'PTX', 'PVB', 'PVC', 'PVD', 'PVO', 'PVS', 'PVY', 'SHE', 'TDG', 'TLP', 'AAA', 'ABS', 'ACM', 'AMC', 'APC', 'APH', 'APP', 'ATG', 'BCA', 'BCB', 'BFC', 'BKC', 'BKG', 'BMC', 'BMJ', 'BMP', 'BRC', 'BRR', 'BT1', 'BVG', 'CAP', 'CBI', 'CKA', 'CKD', 'CLM', 'CMI', 'CPC', 'CST', 'CSV', 'DAG', 'DCM', 'DDV', 'DFC', 'DGC', 'DHB', 'DHC', 'DHM', 'DOC', 'DPM', 'DPR', 'DRG', 'DRI', 'DTL', 'DTT', 'FCM', 'FRC', 'FRM', 'GER', 'GLC', 'GVR', 'GVT', 'HAI', 'HAP', 'HCD', 'HGM', 'HHP', 'HII', 'HLA', 'HLC', 'HMC', 'HMG', 'HNP', 'HPG', 'HPH', 'HPM', 'HRC', 'HSG', 'HSI', 'HSV', 'HVT', 'IRC', 'ITQ', 'KCB', 'KHD', 'KKC', 'KLM', 'KMT', 'KSB', 'KSH', 'KSQ', 'KSV', 'KVC', 'LAS', 'LCM', 'LMC', 'LNC', 'LTG', 'MDC', 'MDF', 'MEL', 'MGC', 'MIC', 'MIM', 'MSR', 'MTA', 'MVB', 'NBC', 'NFC', 'NHH', 'NKG', 'NSG', 'NSH', 'NTP', 'PAS', 'PAT', 'PCE', 'PCH', 'PEC', 'PGN', 'PHR', 'PIS', 'PLC', 'PLP', 'PMB', 'POM', 'PRT', 'PSE', 'PSW', 'QBS', 'RBC', '

In [18]:
direct_cash_flow_url = r'E:\Downloads\master-main\master-main\raw\financial_report_cashflow_direct_yearly_final\financial_report_cashflow_direct_yearly_final.csv'
direct_cash_flow_data=pd.read_csv(direct_cash_flow_url)
direct_cash_flow_data

,ticker,response,table,crawl_datime
0,ABW,{},financial_report_cashflow_direct_yearly_final,2402110040
1,ACB,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
2,APG,"{""Appendix"":{""0"":""5. Thay \u0111\u1ed5i t\u00e...",financial_report_cashflow_direct_yearly_final,2402110040
3,ABT,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
4,AMV,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
...,...,...,...,...
1579,TR1,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1580,ILS,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1581,PBC,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1582,PCM,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040


In [19]:
direct_cash_flow_data_dict = organize_bs_data(direct_cash_flow_data)

No 'Appendix' found for ticker ABW at row 0
No 'Appendix' found for ticker ART at row 7
No 'Appendix' found for ticker BCR at row 72
No 'Appendix' found for ticker BCO at row 73
No 'Appendix' found for ticker DMS at row 348
No 'Appendix' found for ticker DLM at row 349
No 'Appendix' found for ticker PRE at row 1009
No 'Appendix' found for ticker THM at row 1203
No 'Appendix' found for ticker VCI at row 1388


In [20]:
indirect_cash_flow_url = r'E:\Downloads\master-main\master-main\raw\financial_report_cashflow_indirect_yearly_final\financial_report_cashflow_indirect_yearly_final.csv'
indirect_cash_flow_data=pd.read_csv(indirect_cash_flow_url)
indirect_cash_flow_data_dict= organize_bs_data(indirect_cash_flow_data)

No 'Appendix' found for ticker ART at row 0
No 'Appendix' found for ticker ABW at row 2
No 'Appendix' found for ticker BCO at row 70
No 'Appendix' found for ticker BCR at row 71
No 'Appendix' found for ticker DMS at row 347
No 'Appendix' found for ticker DLM at row 348
No 'Appendix' found for ticker PRE at row 1015
No 'Appendix' found for ticker THM at row 1214
No 'Appendix' found for ticker VCI at row 1394


In [21]:
direct_cash_flow_data_dict['GVT']

{'Appendix': {'0': 'I. Lưu chuyển tiền từ hoạt động kinh doanh',
  '1': '1. Tiền thu từ bán hàng, cung cấp dịch vụ và doanh thu khác',
  '2': '2. Tiền chi trả cho người cung cấp hàng hóa và dịch vụ',
  '3': '3. Tiền chi trả cho người lao động',
  '4': '4. Tiền lãi vay đã trả',
  '5': '5. Thuế thu nhập doanh nghiệp đã nộp',
  '6': '6. Tiền thu khác từ hoạt động kinh doanh',
  '7': '7. Tiền chi khác cho hoạt động kinh doanh',
  '8': 'Lưu chuyển tiền thuần từ hoạt động kinh doanh',
  '9': 'II. Lưu chuyển tiền từ hoạt động đầu tư',
  '10': '1.Tiền chi để mua sắm, xây dựng TSCĐ và các tài sản dài hạn khác',
  '11': '2.Tiền thu từ thanh lý, nhượng bán TSCĐ và các tài sản dài hạn khác',
  '12': '3.Tiền chi cho vay, mua các công cụ nợ của đơn vị khác',
  '13': '4.Tiền thu hồi cho vay, bán lại các công cụ nợ của đơn vị khác',
  '14': '5.Tiền chi đầu tư góp vốn vào đơn vị khác',
  '15': '6.Tiền thu hồi đầu tư góp vốn vào đơn vị khác',
  '16': '7.Tiền thu lãi cho vay, cổ tức và lợi nhuận được chi

In [22]:
direct_cash_flow_data

,ticker,response,table,crawl_datime
0,ABW,{},financial_report_cashflow_direct_yearly_final,2402110040
1,ACB,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
2,APG,"{""Appendix"":{""0"":""5. Thay \u0111\u1ed5i t\u00e...",financial_report_cashflow_direct_yearly_final,2402110040
3,ABT,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
4,AMV,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
...,...,...,...,...
1579,TR1,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1580,ILS,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1581,PBC,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040
1582,PCM,"{""Appendix"":{""0"":""I. L\u01b0u chuy\u1ec3n ti\u...",financial_report_cashflow_direct_yearly_final,2402110040


In [23]:
income_structure=pd.read_excel('E:\Downloads\master-main\Income Statement Structure_Official.xlsx',sheet_name='Standard')
balance_sheet_structure=pd.read_excel('E:\Downloads\master-main\Balance Sheet Structure_Official.xlsx',sheet_name='Standard')

In [24]:
# Using the sample data structure to extract the latest data
latest_bs_data = extract_latest_data(bs_data_dict)
latest_income_data = extract_latest_data(income_data_dict)

In [25]:
latest_income_data['DGW']['18. Lợi nhuận sau thuế thu nhập doanh nghiệp(60=50-51-52)']='683,783,287,082'

In [26]:
# Assuming the structure has one column for the standard name and another for variants
# We'll transform this structure into a mapping dictionary
new_mapping_dict = {}

# Iterate through each row, treating the first column as the standard name
# and all other columns as containing variants.
for index, row in income_structure.iterrows():
    standard_name = row.iloc[0]  # The standard name is in the first column
    
    # Iterate through all other columns in this row to find variant names
    for variant in row[1:]:  # Skip the first column which is the standard name
        if pd.notnull(variant):  # Check if the cell is not empty
            new_mapping_dict[variant.strip()] = standard_name

for index, row in balance_sheet_structure.iterrows():
    standard_name = row.iloc[0]  # The standard name is in the first column
    
    # Iterate through all other columns in this row to find variant names
    for variant in row[1:]:  # Skip the first column which is the standard name
        if pd.notnull(variant):  # Check if the cell is not empty
            new_mapping_dict[variant.strip()] = standard_name

In [27]:
mapped_income_data_df = pd.DataFrame()

In [28]:
# Example usage:
current_asset='Current Assets and Short-term Investments'
# Define the list of metric names you want to add as columns
income_metric_names_to_add = ['Profit Before Tax', 'Profit After Tax','Gross Revenue','Current Income Tax Expense','Deferred Income Tax Expense','Interest Expense',]
bs_metric_names_to_add =['Equity','Total Assets','Inventories',current_asset,'Liabilities',"Inventories","Trade and Other Receivables","Long-term Receivables",
                         'Short-term Liabilities','Long-term Liabilities']
# Assuming latest_income_data and mock_mapping_dict are already defined
# Call the function with the list of metrics
company_finance_df = map_multiple_financial_metrics_to_df(latest_income_data, new_mapping_dict, income_metric_names_to_add)
company_finance_df = map_multiple_financial_metrics_to_df(latest_bs_data, new_mapping_dict, bs_metric_names_to_add,company_finance_df)

company_finance_df = company_finance_df.replace('', np.nan)
company_finance_df = company_finance_df.replace(',', '', regex=True).astype(float)

company_finance_df=company_finance_df.reset_index()
company_finance_df.rename(columns={'index':'ticker'},inplace=True)
company_finance_df

,ticker,Profit Before Tax,Profit After Tax,Gross Revenue,Current Income Tax Expense,Deferred Income Tax Expense,Interest Expense,Equity,Total Assets,Inventories,Current Assets and Short-term Investments,Liabilities,Trade and Other Receivables,Long-term Receivables,Short-term Liabilities,Long-term Liabilities
0,ABI,2.764625e+11,2.217984e+11,NaN,NaN,NaN,NaN,1.471503e+12,3.575994e+12,1.785164e+09,3.386741e+12,2.104491e+12,1.478013e+11,1.021534e+10,3.082843e+11,NaN
1,AMV,5.810098e+10,5.641300e+10,2.848445e+11,1.687979e+09,NaN,1.961624e+10,1.675538e+12,2.000041e+12,2.383056e+11,1.480361e+12,3.245038e+11,1.154135e+12,1.034286e+11,1.821392e+11,1.423645e+11
2,ALV,5.725733e+09,4.534117e+09,1.052871e+11,1.191616e+09,NaN,1.036166e+09,6.382208e+10,9.668934e+10,1.492936e+09,4.320797e+10,3.286726e+10,4.137441e+10,4.500000e+10,3.286726e+10,NaN
3,APL,3.046871e+09,2.053039e+09,5.301633e+11,9.938320e+08,NaN,4.539645e+09,1.387666e+10,1.428454e+11,4.935781e+10,1.374072e+11,1.289688e+11,8.442780e+10,NaN,1.289688e+11,NaN
4,AAT,1.120193e+11,8.902814e+10,9.616258e+11,2.299113e+10,NaN,1.922840e+10,7.398949e+11,1.056918e+12,1.573822e+10,4.472210e+11,3.170227e+11,4.141493e+11,6.645000e+10,1.357298e+11,1.812929e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,IRC,1.142030e+10,8.939199e+09,1.632926e+10,2.481102e+09,NaN,9.972603e+06,1.834082e+11,1.934509e+11,2.555332e+10,1.295348e+11,1.004268e+10,1.396767e+09,NaN,1.004268e+10,NaN
1567,LAW,1.535529e+10,1.379468e+10,2.471961e+11,1.560603e+09,NaN,5.303422e+09,1.683237e+11,3.278870e+11,2.500844e+10,4.211102e+10,1.595633e+11,1.101811e+10,NaN,7.044761e+10,8.911569e+10
1568,VIH,1.007310e+10,6.401181e+09,6.149261e+11,3.671921e+09,NaN,1.791394e+10,3.250919e+10,4.028766e+11,1.156250e+11,1.891045e+11,3.703674e+11,4.300644e+10,NaN,3.090603e+11,6.130706e+10
1569,PAT,1.014138e+12,9.632532e+11,3.149890e+12,5.088452e+10,NaN,2.942013e+09,7.818513e+11,1.194319e+12,6.713912e+10,7.877156e+11,4.124676e+11,1.597508e+11,2.000000e+09,4.124676e+11,NaN


In [29]:
beta_df=pd.read_csv(r'E:\Downloads\master-main\beta.csv')

price_url=r'E:\Downloads\master-main\master-main\curated\curated_stock_ohlc.csv'
company_profile_url=r'E:\Downloads\master-main\master-main\curated\curated_company_profile.csv'
company_profile_data=pd.read_csv(company_profile_url).drop_duplicates()

price_data=pd.read_csv(price_url)
price_data=price_data[['time','close','volume','ticker']]

latest_price_data = price_data.sort_values(by=['ticker', 'time']).groupby('ticker').last().reset_index()[['ticker', 'close']]
if 'Close Price' in company_finance_df:
    pass
else:
    company_finance_df=company_finance_df.merge(latest_price_data,how='left',on='ticker')
    company_finance_df.rename(columns={'close':"Close Price"},inplace=True)

if 'L1N'in company_finance_df:
    pass
else:
    company_finance_df=company_finance_df.merge(company_industry_df[['Stock','L1N','L2N','L3N','L4N']],how='left',left_on='ticker',right_on='Stock')
    company_finance_df.drop(columns=['Stock'],inplace=True)

if "Number of outstanding shares" in company_finance_df:
    pass
else:
    company_finance_df=company_finance_df.merge(company_profile_data[['ticker','outstandingShare']],how='left',on='ticker')
    company_finance_df['outstandingShare']=company_finance_df['outstandingShare']*1000000
    company_finance_df.rename(columns={'outstandingShare':"Number of outstanding shares"},inplace=True)

if 'beta' in company_finance_df:
    pass
else:
    company_finance_df=company_finance_df.merge(beta_df,how='left',on='ticker')

In [32]:
company_finance_df['Tax Expense']=company_finance_df['Current Income Tax Expense']+company_finance_df['Deferred Income Tax Expense']

company_finance_df['EBIT']=company_finance_df['Profit Before Tax'] + company_finance_df['Interest Expense']


company_finance_df['ROE']=company_finance_df['Profit After Tax']/company_finance_df['Equity']
company_finance_df['ROA']=company_finance_df['Profit After Tax']/company_finance_df['Total Assets']

company_finance_df['Receivable Turnover']=company_finance_df['Gross Revenue']/company_finance_df['Trade and Other Receivables']
company_finance_df['Inventory Turnover']=company_finance_df['Gross Revenue']/company_finance_df['Inventories']
company_finance_df['CA Turnover']=company_finance_df['Gross Revenue']/company_finance_df[current_asset]
company_finance_df['TA Turnover']=company_finance_df['Gross Revenue']/company_finance_df['Total Assets']


company_finance_df['Current Ratio']=company_finance_df[current_asset]/company_finance_df['Short-term Liabilities']
company_finance_df['Interest Coverage Ratio']=company_finance_df['EBIT']/company_finance_df['Interest Expense']

company_finance_df['Leverage']=company_finance_df['Liabilities']/company_finance_df['Equity']
company_finance_df['Short-term/Total Liabilities']=company_finance_df['Short-term Liabilities']/company_finance_df['Liabilities']

company_finance_df['P/S']=company_finance_df['Close Price']/(company_finance_df['Gross Revenue']/company_finance_df['Number of outstanding shares'])
company_finance_df['P/B']=company_finance_df['Close Price']/(company_finance_df['Equity']/company_finance_df['Number of outstanding shares'])

#Industry
# Rank by Gross Revenue within L1, L2, L3, and L4 categories (assuming L columns are not null)
def rank_within_group(df, group_cols, rank_col):
    """
    Ranks data within groups defined by multiple columns.

    Args:
        df (pandas.DataFrame): The dataframe containing the data.
        group_cols (list): A list of column names to group by.
        rank_col (str): The column name to rank by.

    Returns:
        None (Modifies the input dataframe in-place)
    """
    # Sort by group columns and ranking column
    df.sort_values(by=group_cols + [rank_col], ascending=True, inplace=True)

    # Assign ranks within each group
    for col in group_cols:
        df['Rank ' + col] = df.groupby(col)[rank_col].transform('rank',ascending=False)


rank_within_group(company_finance_df, ['L1N', 'L2N', 'L3N', 'L4N'], 'Gross Revenue')

#CAPM
company_finance_df['Required rate of return'] = company_finance_df['L1N'].map(weighted_average_roe(company_finance_df.copy(), 'L1N', 'Gross Revenue', 'ROE', 5))

company_finance_df['Discount Rate'] = np.maximum(
                                        risk_free_rate,
                                        risk_free_rate + company_finance_df['beta'] * (company_finance_df['Required rate of return'] - risk_free_rate))

company_finance_df['Discount Time Window (year)']=np.log(2)/np.log(1+company_finance_df['Discount Rate'])

company_finance_df['SaleGrowth (TTM)']=5/100
company_finance_df['SalesGrowth_03Yr']=5/100
company_finance_df['Projected SalesGrowth']=5/100
company_finance_df['PreTaxMargin (TTM)']=10/100
company_finance_df['Projected_ProfitPreTax']=company_finance_df['Gross Revenue']*(1+company_finance_df['Projected SalesGrowth'])*np.maximum(0,company_finance_df['PreTaxMargin (TTM)'])
company_finance_df['Projected_ProfitAfterTax']=company_finance_df['Projected_ProfitPreTax']*(1-income_tax) 
company_finance_df['CAPM']=company_finance_df['Projected_ProfitAfterTax']/(company_finance_df['Discount Rate'])/(1+company_finance_df['Discount Rate'])**company_finance_df['Discount Time Window (year)']

company_finance_df['Asset Quality Discount']=0 #cần tình
company_finance_df['Discounted Amount']=(company_finance_df['Inventories']+company_finance_df['Trade and Other Receivables'])*company_finance_df['Asset Quality Discount']
company_finance_df['Corrected Equity (BV)']=company_finance_df['Equity']-company_finance_df['Discounted Amount'] 
company_finance_df['Total_PresentValues']=company_finance_df['CAPM']+company_finance_df['Corrected Equity (BV)']
company_finance_df['Adjusted CAPM Per Share']=company_finance_df['Total_PresentValues']/company_finance_df['Number of outstanding shares']

In [33]:
company_finance_df.columns

Index(['ticker', 'Profit Before Tax', 'Profit After Tax', 'Gross Revenue',
       'Current Income Tax Expense', 'Deferred Income Tax Expense',
       'Interest Expense', 'Equity', 'Total Assets', 'Inventories',
       'Current Assets and Short-term Investments', 'Liabilities',
       'Trade and Other Receivables', 'Long-term Receivables',
       'Short-term Liabilities', 'Long-term Liabilities', 'Close Price', 'L1N',
       'L2N', 'L3N', 'L4N', 'Number of outstanding shares', 'Unnamed: 0',
       'beta', 'Tax Expense', 'EBIT', 'ROE', 'ROA', 'Receivable Turnover',
       'Inventory Turnover', 'CA Turnover', 'TA Turnover', 'Current Ratio',
       'Interest Coverage Ratio', 'Leverage', 'Short-term/Total Liabilities',
       'P/S', 'P/B', 'Rank L1N', 'Rank L2N', 'Rank L3N', 'Rank L4N',
       'Required rate of return', 'Discount Rate',
       'Discount Time Window (year)', 'SaleGrowth (TTM)', 'SalesGrowth_03Yr',
       'Projected SalesGrowth', 'PreTaxMargin (TTM)', 'Projected_ProfitPreTax'

In [34]:
# Example of usage for one level, you can concatenate the results for all levels
# industry_summary_df = calculate_weighted_averages(company_finance_df, 'L1N')
# For all levels, you might concatenate like this:
all_levels = ['L1N', 'L2N', 'L3N', 'L4N']
industry_summary_df = pd.concat([calculate_weighted_averages(company_finance_df, level) for level in all_levels])


In [35]:
# Placeholder list to collect DataFrames for each level
level_dfs = []

# Function to process each level
def process_level(df, level):
    # Get unique industries for the level
    industries = df[level].unique()
    
    # Placeholder list for processed industries
    industry_dfs = []
    
    # Process each industry
    for industry in industries:
        # Filter the DataFrame for the industry and get the top 5 companies
        industry_df = df[df[level] == industry].nsmallest(5, f'Rank {level}')
        
        # Get the list of top 5 company tickers
        top_companies = industry_df['ticker'].tolist()
        
        # Get the metrics for the top 5 companies
        metrics = industry_df[['ROE', 'ROA', 'Receivable Turnover', 'Inventory Turnover', 
                               'CA Turnover', 'TA Turnover', 'Current Ratio', 
                               'Interest Coverage Ratio', 'Leverage', 'Short-term/Total Liabilities', 
                               'P/S', 'P/B']]
        
        # Create a new DataFrame for the industry with the required structure
        data = {
            'Industry Name': industry,
            'Level': level[-2],  # Extract the numeric part of L1N, L2N, etc.
            'Top 5 Companies': ", ".join(top_companies)
        }
        
        # Add the metrics to the data dictionary
        for metric in metrics:
            data[metric] = industry_df[metric].mean()  # Calculate summarized metric, here we take the mean as example
        
        # Convert the data dictionary to a DataFrame
        industry_summary_df = pd.DataFrame([data])
        
        # Append to the list of processed industries
        industry_dfs.append(industry_summary_df)
    
    # Concatenate all industries for the level
    return pd.concat(industry_dfs, ignore_index=True)

# Process each level and concatenate results
for level in ['L4N', 'L3N', 'L2N', 'L1N']:  # Start from L4N down to L1N
    level_df = process_level(company_finance_df, level)
    level_dfs.append(level_df)

# Concatenate all levels into a final DataFrame
final_summary_df = pd.concat(level_dfs, ignore_index=True)
final_summary_df

,Industry Name,Level,Top 5 Companies,ROE,ROA,Receivable Turnover,Inventory Turnover,CA Turnover,TA Turnover,Current Ratio,Interest Coverage Ratio,Leverage,Short-term/Total Liabilities,P/S,P/B
0,Cung cấp nước sạch,4,"BWE, CLW, DNW, TDW, HPW",0.158836,0.080006,32.935435,33.732049,3.277706,1.297188,1.522678,9.903109,1.014648,0.484948,NaN,NaN
1,Phân phối khí đốt,4,"GAS, PGD, PEG, PGS, PVG",0.128772,0.070054,13.616234,119.112386,6.861494,3.260030,1.763681,551.227387,1.181497,0.846089,NaN,NaN
2,Đa tiện ích,4,"SDV, SZE, MQN, HEP, DNE",0.086912,0.039258,7.020522,99.570204,2.371121,1.103254,1.037272,51.554523,1.252784,0.844114,0.207405,NaN
3,Sản xuất và cung cấp điện truyền thống,4,"PGV, POW, DTK, QTP, NT2",0.128167,0.069355,3.464742,25.228057,2.200464,0.872841,1.521487,16.626921,1.092649,0.697256,0.624209,1.054125
4,Công nghiệp đa ngành,4,"DLT, SMA",0.105715,0.041771,7.051686,29.054475,4.006316,1.243257,0.580515,2.863151,5.013425,0.907317,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Tài chính,1,"VIC, VHM, ASM, FLC, PVI",0.082682,0.031977,1.435654,1105.288879,0.717093,0.399798,1.074043,19.312814,2.398507,0.782836,NaN,NaN
154,Viễn thông,1,"FOX, ABC, FOC, TTN",0.529283,0.369412,7.062946,379.903423,1.469672,1.049713,3.099446,56.191774,0.555976,0.946461,0.728422,0.641643
155,Vật liệu cơ bản,1,"HPG, HSG, TVN, GVR, TMB",0.114955,0.035713,55.082386,7.596605,4.649624,3.353094,1.474184,4.727702,1.482299,0.800503,1.463625,1.213042
156,Y tế,1,"VMD, DVN, DHG, CDP, TRA",0.135478,0.078995,6.353030,5.165483,1.773794,1.421721,2.278676,88.658653,3.373889,0.972401,1.369873,2.396732


In [37]:
groups = {
    'Market': ['P/B', 'P/S'],
    'Cost Profit': ['ROE', 'ROA'],
    'Activity Indicators': ['Receivable Turnover', 'Inventory Turnover', 'CA Turnover', 'TA Turnover'],
    'Liquidity Indicators': ['Current Ratio', 'Interest Coverage Ratio'],
    'Leverage':['Leverage','Short-term/Total Liabilities']
}

health_groups={'Cost Profit': ['ROE', 'ROA'],
    'Activity Indicators': ['Receivable Turnover', 'Inventory Turnover', 'CA Turnover', 'TA Turnover'],
    'Liquidity Indicators': ['Current Ratio', 'Interest Coverage Ratio'],
    'Leverage':['Leverage','Short-term/Total Liabilities']
}

# Flatten the group structure for easier data retrieval
metrics_columns = [metric for group in groups.values() for metric in group]
health_metrics_columns = [metric for group in health_groups.values() for metric in group]
health_metrics_columns_negative = ['Leverage']

In [38]:
# Usage example would remain the same
ticker_data_df = build_company_metrics_dataframe('VHM', company_finance_df, final_summary_df)
ticker_data_df

Market  \
                                                                        P/B   
Level Industry                                      Company                   
1     Tài chính                                     Industry Average    NaN   
                                                    VIC                 NaN   
                                                    VHM                 NaN   
                                                    ASM                 NaN   
                                                    FLC                 NaN   
                                                    PVI                 NaN   
2     Bất động sản                                  Industry Average    NaN   
                                                    VIC                 NaN   
                                                    VHM                 NaN   
                                                    ASM                 NaN   
                                                    FLC                 NaN   
                                                    NVL                 NaN   
3     Đầu tư bất động sản và dịch vụ                Industry Average    NaN   
                                                    VIC                 NaN   
                                                    VHM                 NaN   
                                                    ASM                 NaN   
                                                    FLC                 NaN   
                                                    NVL                 NaN   
4     Các công ty đầu cơ và phát triển bất động sản Industry Average    NaN   
                                                    VIC                 NaN   
                                                    VHM                 NaN   
                                                    ASM                 NaN   
                                                    NVL                 NaN   
                                                    IDC                 NaN   

                                                                          \
                                                                     P/S   
Level Industry                                      Company                
1     Tài chính                                     Industry Average NaN   
                                                    VIC              NaN   
                                                    VHM              NaN   
                                                    ASM              NaN   
                                                    FLC              NaN   
                                                    PVI              NaN   
2     Bất động sản                                  Industry Average NaN   
                                                    VIC              NaN   
                                                    VHM              NaN   
                                                    ASM              NaN   
                                                    FLC              NaN   
                                                    NVL              NaN   
3     Đầu tư bất động sản và dịch vụ                Industry Average NaN   
                                                    VIC              NaN   
                                                    VHM              NaN   
                                                    ASM              NaN   
                                                    FLC              NaN   
                                                    NVL              NaN   
4     Các công ty đầu cơ và phát triển bất động sản Industry Average NaN   
                                                    VIC              NaN   
                                                    VHM              NaN   
                                                    ASM              NaN   
                                   

In [88]:
health_evaluation = {}

for ticker in company_industry_df['Stock']:
    
    # Attempt to get the benchmark industry for the ticker
    filtered_df = company_finance_df[company_finance_df['ticker'] == ticker]
    if not filtered_df.empty:
        benchmark_industry = filtered_df['L4N'].values[0]
        health_evaluation[ticker] = {ticker:{},'Benchmark': {}, 'Score': {}}

        for metric in health_metrics_columns:
            # Initialize metric values as None
            ticker_metric, benchmark_metric = None, None
            
            # Attempt to get the metric value for the ticker
            if metric in filtered_df.columns and not filtered_df[metric].isnull().all():
                ticker_metric = filtered_df[metric].dropna().values[0]

            # Attempt to get the benchmark metric value for the industry
            benchmark_df = industry_summary_df[
                (industry_summary_df['Industry Level'] == 'L4N') & 
                (industry_summary_df['Industry Name'] == benchmark_industry)
            ]
            if not benchmark_df.empty and metric in benchmark_df.columns and not benchmark_df[metric].isnull().all():
                benchmark_metric = benchmark_df[metric].dropna().values[0]

            # Comparison and scoring logic
            score = None
            if ticker_metric is not None and benchmark_metric is not None:
                if metric in health_metrics_columns_negative:
                    if ticker_metric <= benchmark_metric / indicator_range_max:
                        score = 1
                    elif ticker_metric < benchmark_metric *  indicator_range_max:
                        score = score_modifier_max
                    elif ticker_metric < benchmark_metric / indicator_range_min:
                        score = score_modifier_min
                    else:
                        score = 0
                else:
                    if ticker_metric >= benchmark_metric * indicator_range_max:
                        score = 1
                    elif ticker_metric > benchmark_metric / indicator_range_max:
                        score = score_modifier_max
                    elif ticker_metric > benchmark_metric * indicator_range_min:
                        score = score_modifier_min
                    else:
                        score = 0
            
            # Assigning metric values and scores
            health_evaluation[ticker][ticker][metric] = ticker_metric
            health_evaluation[ticker]['Benchmark'][metric] = benchmark_metric
            health_evaluation[ticker]['Score'][metric] = score
    else:
        # Handle the case where the ticker does not match any row in company_finance_df
        health_evaluation[ticker] = {'Error': 'Ticker not found in the finance DataFrame'}

In [96]:
health_evaluation['VNM']['Score']

{'ROE': 1,
 'ROA': 1,
 'Receivable Turnover': 1,
 'Inventory Turnover': 1,
 'CA Turnover': 0.8,
 'TA Turnover': 1,
 'Current Ratio': 1,
 'Interest Coverage Ratio': 1,
 'Leverage': 1,
 'Short-term/Total Liabilities': 1}

In [101]:
from statistics import mean

# Calculate total scores if needed
for ticker, data in health_evaluation.items():
    health_evaluation[ticker]['Total Score'] = {}
    if 'Score' in data:
        total_group_scores = []
        for group_name, metrics_list in health_groups.items():
            group_scores = [score for score in [data['Score'].get(metric) for metric in metrics_list] if score is not None]
            if group_scores:  # Check if there are scores available for the group
                group_average_score = mean(group_scores)
            else:
                group_average_score = None
            total_group_scores.append(group_average_score)
            health_evaluation[ticker]['Total Score'][group_name] = group_average_score
        
        # Calculate the total score as the sum of group scores
        total_group_scores = [score for score in total_group_scores if score is not None]  # Filter out None values
        if total_group_scores:
            health_evaluation[ticker]['Total Score']['Total Score'] = sum(total_group_scores)

{'Cost Profit': 0.25,
 'Activity Indicators': 0.575,
 'Liquidity Indicators': 0.25,
 'Leverage': 0.5,
 'Total Score': 1.575}

In [50]:
def transform_to_dataframe(health_data, health_groups):
    ticker=list(health_data.keys())[0]
    # Prepare the multi-index for columns
    tuples = [(group, metric) for group, metrics in health_groups.items() for metric in metrics]
    multi_index = pd.MultiIndex.from_tuples(tuples, names=['Health Valuation', 'Metric'])

    # Prepare the data for the DataFrame
    # Rows will contain data for Ticker, Benchmark, and Score in that order
    data = {
        ticker: [],
        'Benchmark': [],
        'Score': []
    }

    for group, metrics in health_groups.items():
        for metric in metrics:
            data[ticker].append(health_data[ticker].get(metric, None))
            data['Benchmark'].append(health_data['Benchmark'].get(metric, None))
            data['Score'].append(health_data['Score'].get(metric, None))

    # Create the DataFrame with a list of the data dictionaries
    df = pd.DataFrame([data[ticker], data['Benchmark'], data['Score']], 
                      index=[ticker, 'Benchmark', 'Score'],
                      columns=multi_index)
    cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
    }
    index_names = {
        'selector': '.index_name',
        'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
    }
    headers = {
        'selector': 'th:not(.index_name)',
        'props': 'background-color: #000066; color: white;'
    }
    df=df.style.format('{:.2f}')
    df.set_table_styles([cell_hover, index_names, headers])
    df.set_table_styles([
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;text-align:center'}], overwrite=False)

    return df

transform_to_dataframe(health_evaluation['VNM'], health_groups)

In [103]:
company_finance_df['Health Score'] = company_finance_df['ticker'].apply(lambda x: health_evaluation.get(x, {}).get('Total Score', {}).get('Total Score', None))

In [52]:
industry_summary_df

,Industry Name,Industry Level,ROE,ROA,Receivable Turnover,Inventory Turnover,CA Turnover,TA Turnover,Current Ratio,Interest Coverage Ratio,Leverage,Short-term/Total Liabilities,P/S,P/B
0,Các dịch vụ hạ tầng,L1N,0.190923,0.117129,5.136517,37.029087,1.993372,1.073001,2.956869,NaN,1.070965,0.559506,NaN,NaN
1,Công nghiệp,L1N,0.097119,0.032448,9.190564,16.278413,2.504024,1.603570,1.139048,3.895555,1.999817,0.735190,NaN,NaN
2,Công nghệ,L1N,0.170569,0.084603,3.914308,19.209541,1.263970,0.748086,1.504780,9.876802,1.033993,0.860797,NaN,NaN
3,Dầu khí,L1N,0.125113,0.069146,17.995296,18.418906,4.790484,3.366609,1.469274,20.920791,1.334259,0.963543,NaN,NaN
4,Dịch vụ tiêu dùng,L1N,0.358110,-0.003490,29.576245,14.697002,3.513988,2.000734,1.190873,-33.638207,-0.007948,0.800027,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Đa tiện ích,L4N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Đào tạo nhân sự và việc làm,L4N,0.381743,0.227892,12.951734,1686.706633,3.618404,2.793037,2.422982,27.156713,0.654287,0.799688,NaN,NaN
78,Đường sắt,L4N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.651365,NaN,NaN,NaN,NaN
79,Đồ điện tử tiêu dùng,L4N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
def calculate_PB_valuation(row):
    global industry_summary_df
    industry=row['L1N']
    equity_book_value_per_share=row['Equity']/row['Number of outstanding shares']

    filtered_df = industry_summary_df[(industry_summary_df['Industry Level']=='L1N') &
                                  (industry_summary_df['Industry Name']==industry)]
    if not filtered_df.empty:
        industry_PB = filtered_df['P/B'].values[0]
    else:
        # Handle the case where no matching industry is found (e.g., assign a default value)
        industry_PB = np.nan
    price=equity_book_value_per_share*industry_PB
    return price

company_finance_df['PB Valuation'] = company_finance_df.apply(calculate_PB_valuation, axis=1)

def calculate_PS_valuation(row):
    global industry_summary_df
    industry=row['L4N']
    sales_per_share=row['Gross Revenue']/row['Number of outstanding shares']

    filtered_df = industry_summary_df[(industry_summary_df['Industry Level']=='L4N') &
                                  (industry_summary_df['Industry Name']==industry)]
    if not filtered_df.empty:
        industry_PS = filtered_df['P/S'].values[0]
    else:
        # Handle the case where no matching industry is found (e.g., assign a default value)
        industry_PS = np.nan
    price=sales_per_share*industry_PS
    return price
company_finance_df['PS Valuation'] = company_finance_df.apply(calculate_PS_valuation, axis=1)

In [65]:
company_finance_df['Fair Price']=(company_finance_df['Adjusted CAPM Per Share']+company_finance_df['PB Valuation']+company_finance_df['PS Valuation'])/3
company_finance_df['Opportunity']=company_finance_df['Close Price']/company_finance_df['Fair Price']-1

company_finance_df

,ticker,Profit Before Tax,Profit After Tax,Gross Revenue,Current Income Tax Expense,Deferred Income Tax Expense,Interest Expense,Equity,Total Assets,Inventories,...,CAPM,Asset Quality Discount,Discounted Amount,Corrected Equity (BV),Total_PresentValues,Adjusted CAPM Per Share,PB Valuation,PS Valuation,Fair Price,Opportunity
171,BWA,1.887714e+09,1.431363e+09,3.205572e+10,4.563509e+08,NaN,NaN,NaN,NaN,NaN,...,1.923343e+10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,DVW,1.140112e+10,9.944885e+09,5.047155e+10,1.456238e+09,NaN,5.663810e+08,NaN,NaN,NaN,...,2.599062e+10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,LKW,1.884673e+10,1.664225e+10,5.283414e+10,2.204481e+09,NaN,NaN,NaN,NaN,NaN,...,3.170048e+10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,GLW,8.106389e+09,6.430401e+09,5.461556e+10,1.675988e+09,NaN,NaN,1.961324e+11,2.025229e+11,3.611563e+09,...,3.276934e+10,0,0.0,1.961324e+11,2.289017e+11,NaN,NaN,NaN,NaN,NaN
117,BLW,1.034674e+10,8.973589e+09,6.376446e+10,1.373153e+09,NaN,NaN,1.161739e+11,1.266639e+11,5.663133e+09,...,3.338171e+10,0,0.0,1.161739e+11,1.495557e+11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,XDC,9.567207e+09,7.648246e+09,2.794160e+11,1.918961e+09,NaN,3.550555e+08,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,KSS,6.636000e+09,3.949953e+09,3.303413e+11,2.686048e+09,NaN,8.925600e+10,1.725675e+11,1.882796e+12,3.464821e+11,...,NaN,0,0.0,1.725675e+11,NaN,NaN,NaN,NaN,NaN,NaN
1112,SDT,-2.267002e+09,-1.564921e+10,4.449503e+11,8.373672e+09,5.008534e+09,8.786088e+10,8.135169e+11,2.217641e+12,3.715516e+11,...,NaN,0,0.0,8.135169e+11,NaN,NaN,NaN,NaN,NaN,NaN
1245,TKC,4.091072e+09,3.233672e+09,6.065558e+11,6.855924e+09,-5.998524e+09,4.926956e+10,-2.789166e+11,1.440108e+12,1.840733e+11,...,NaN,0,0.0,-2.789166e+11,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
income_structure

,Accounts,General,Banking,Insurance,Stock
0,Gross Revenue,1. Doanh thu bán hàng và cung cấp dịch vụ,NaN,NaN,NaN
1,Revenue Deductions,2. Các khoản giảm trừ doanh thu,NaN,NaN,NaN
2,Net Revenue,3. Doanh thu thuần về bán hàng và cung cấp dịc...,NaN,NaN,NaN
3,Cost of Goods Sold,4. Giá vốn hàng bán,NaN,NaN,NaN
4,Gross Profit,5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...,NaN,NaN,NaN
5,Financial Income,6. Doanh thu hoạt động tài chính,NaN,NaN,NaN
6,Financial Expenses,7. Chi phí tài chính,NaN,NaN,NaN
7,Interest Expense,- Trong đó: Chi phí lãi vay,NaN,NaN,NaN
8,Share of Profit or Loss of Associates and Join...,"8. Phần lãi lỗ trong công ty liên doanh, liên kết",Lợi nhuận được hưởng từ các công ty liên kết v...,NaN,NaN
9,Selling Expenses,9. Chi phí bán hàng,NaN,NaN,NaN


In [80]:
# Reset the index to default integer index
#company_finance_df.reset_index(drop=True, inplace=True)

# Number of areas per column
num_areas = 16

# Create a DataFrame where each cell is the percentage of NULL values in its area
null_df = company_finance_df.isnull().astype(int).groupby(company_finance_df.index // (len(company_finance_df) // num_areas)).mean()

# Create a heatmap
plt.figure(figsize=(25,15))  # Increase figure size
sns.heatmap(null_df, cmap='RdYlGn_r', cbar=True)#, annot=True, fmt=".1%", annot_kws={"size": 8})  # Decrease font size
plt.title('NULL Values Density Heatmap')
plt.xlabel('Columns')
plt.ylabel('Area')
plt.show()